#  Building a simple statistic language model on a shakespeare courpus


This is a simple python code to show how to build a n-gram language model.
Here are the steps:

1. Preprocessig
      1. Get raw text
      1. Tokenize the text
      1. Clean the text
2. Building the n-gram lm (language model)
      1. get a sequence of word
      2. get the frequency of the n-grams.
      3. Loop through the whole corpus, store and compute the possibility of the existing n-grams.


Let's start now.
The first thing is get text from a txt file. One thing should be concerned is the encoding. 
Since we use python 3, make sure the input file is encoded in unicode.

In [20]:
# store the file line by lines
def get_raw_text(path):
    with open(path) as f:
        return f.readlines()
raw_text = get_raw_text("data/train_shakespeare.txt")

In [21]:
# let see the first line 
print(raw_text[0])

Hence! home, you idle creatures get you home: Is this a holiday? what! know you not, Being mechanical, you ought not walk Upon a labouring day without the sign Of your profession? Speak, what trade art thou?



Looks nice!
Then we can do the tokenization. For English it is rather a simple task because words are seperated by spaces.
However, the punctuation is sticked to the previous word, you should be careful about that.

In [27]:
# Tokenize the text
# Notice that at the end of line we have a newline character, so we have to remove it first by strip()
def split_by_space(text):
    return text.strip().split()

In [28]:
# let see how it works
print(split_by_space(raw_text[0]))

['Hence!', 'home,', 'you', 'idle', 'creatures', 'get', 'you', 'home:', 'Is', 'this', 'a', 'holiday?', 'what!', 'know', 'you', 'not,', 'Being', 'mechanical,', 'you', 'ought', 'not', 'walk', 'Upon', 'a', 'labouring', 'day', 'without', 'the', 'sign', 'Of', 'your', 'profession?', 'Speak,', 'what', 'trade', 'art', 'thou?']


In [49]:
# We don't want the punctuation stick to the word, so lets seperate it from the word


def split_punc(tokens):
    output = []
    punc = '!'
    for token in tokens:
        for i in range(len(token),0,-1):
            if token[:i].isalpha():
                if i==len(token):
                    output.append(token)
                else:
                    output.append(token[:i])
                    output.append(token[i:])
                break
    return output

In [50]:
# let see how it works
print(split_punc(tokenize(raw_text[0])))

['Hence', '!', 'home', ',', 'you', 'idle', 'creatures', 'get', 'you', 'home', ':', 'Is', 'this', 'a', 'holiday', '?', 'what', '!', 'know', 'you', 'not', ',', 'Being', 'mechanical', ',', 'you', 'ought', 'not', 'walk', 'Upon', 'a', 'labouring', 'day', 'without', 'the', 'sign', 'Of', 'your', 'profession', '?', 'Speak', ',', 'what', 'trade', 'art', 'thou', '?']


In [54]:
# Turn uppercase into lowercase

def lowercase(tokens):
    output = []
    for token in tokens:
        output.append(token.lower())
    return output

In [55]:
print(lowercase(split_punc(tokenize(raw_text[0]))))

['hence', '!', 'home', ',', 'you', 'idle', 'creatures', 'get', 'you', 'home', ':', 'is', 'this', 'a', 'holiday', '?', 'what', '!', 'know', 'you', 'not', ',', 'being', 'mechanical', ',', 'you', 'ought', 'not', 'walk', 'upon', 'a', 'labouring', 'day', 'without', 'the', 'sign', 'of', 'your', 'profession', '?', 'speak', ',', 'what', 'trade', 'art', 'thou', '?']


In [67]:
# pack them together
def tokenizer(raw_text):
    tokenized_text = []
    for line in raw_text:
        tokenized_text.append(lowercase(split_punc(tokenize(line))))
    return tokenized_text



In [69]:
print(tokenizer(raw_text[:]))

[['hence', '!', 'home', ',', 'you', 'idle', 'creatures', 'get', 'you', 'home', ':', 'is', 'this', 'a', 'holiday', '?', 'what', '!', 'know', 'you', 'not', ',', 'being', 'mechanical', ',', 'you', 'ought', 'not', 'walk', 'upon', 'a', 'labouring', 'day', 'without', 'the', 'sign', 'of', 'your', 'profession', '?', 'speak', ',', 'what', 'trade', 'art', 'thou', '?'], ['why', ',', 'sir', ',', 'a', 'carpenter', '.'], ['where', 'is', 'thy', 'leather', 'apron', 'and', 'thy', 'rule', '?', 'what', 'dost', 'thou', 'with', 'thy', 'best', 'apparel', 'on', '?', 'you', ',', 'sir', ',', 'what', 'trade', 'are', 'you', '?'], ['truly', ',', 'sir', ',', 'in', 'respect', 'of', 'a', 'fine', 'workman', ',', 'i', 'am', 'but', ',', 'as', 'you', 'would', 'say', ',', 'a', 'cobbler', '.'], ['but', 'what', 'trade', 'art', 'thou', '?', 'answer', 'me', 'directly', '.'], ['a', 'trade', ',', 'sir', ',', 'that', ',', 'i', 'hope', ',', 'i', 'may', 'use', 'with', 'a', 'safe', 'conscience', ';', 'which', 'is', ',', 'indeed', 

In [70]:
print(len(raw_text))

17251
